In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine, text

DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_NAME = os.getenv('DB_NAME')
DB_HOST = 'localhost'
DB_PORT = '5432'

connection_string = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'

engine = create_engine(connection_string)

In [2]:
# 1. Вывести распределение (количество) клиентов по сферам деятельности, отсортировав результат по убыванию количества.
query_1 = """
SELECT
    c.job_industry_category,
    COUNT(*) AS cnt
FROM mipt.customer c
GROUP BY c.job_industry_category
ORDER BY cnt DESC
"""

df_1 = pd.read_sql_query(text(query_1), engine)
df_1

,job_industry_category,cnt
0,Manufacturing,799
1,Financial Services,774
2,n/a,656
3,Health,602
4,Retail,358
5,Property,267
6,IT,223
7,Entertainment,136
8,Argiculture,113
9,Telecommunications,72


In [3]:
# 2. Найти общую сумму дохода (list_price*quantity) по всем подтвержденным заказам за каждый месяц по сферам деятельности клиентов.
# Отсортировать результат по году, месяцу и сфере деятельности.
query_2 = """
SELECT
    c.job_industry_category,
    DATE_TRUNC('month', o.order_date::DATE)::DATE AS month,
    ROUND(SUM(p.list_price * oi.quantity)::NUMERIC, 2) AS amount
FROM mipt.customer c
JOIN mipt.orders o ON c.customer_id = o.customer_id
JOIN mipt.order_items oi ON o.order_id = oi.order_id
JOIN mipt.product p ON oi.product_id = p.product_id
WHERE o.order_status = 'Approved'
GROUP BY
    c.job_industry_category,
    DATE_TRUNC('month', o.order_date::DATE)::DATE
ORDER BY
    month,
    c.job_industry_category
"""

df_2 = pd.read_sql_query(text(query_2), engine)
df_2

,job_industry_category,month,amount
0,Argiculture,2017-01-01,184857.02
1,Entertainment,2017-01-01,274765.58
2,Financial Services,2017-01-01,1694916.21
3,Health,2017-01-01,1396123.62
4,IT,2017-01-01,538294.62
...,...,...,...
115,Manufacturing,2017-12-01,1648110.47
116,n/a,2017-12-01,1437965.86
117,Property,2017-12-01,599560.47
118,Retail,2017-12-01,793873.13


In [6]:
# 4. Найти по всем клиентам: сумму всех заказов (общего дохода), максимум, минимум и количество заказов,
# а также среднюю сумму заказа по каждому клиенту. Отсортировать результат по убыванию суммы всех заказов и количества заказов.
# Выполнить двумя способами: используя только GROUP BY и используя только оконные функции. Сравнить результат.

# GROUP BY version
query_4_groupby = """
SELECT
    c.customer_id,
    ROUND(SUM(oi.quantity * oi.item_list_price_at_sale)::NUMERIC, 2) AS amount,
    MAX(oi.quantity * oi.item_list_price_at_sale) AS max_amount,
    MIN(oi.quantity * oi.item_list_price_at_sale) AS min_amount,
    COUNT(DISTINCT o.order_id) AS cnt_orders,
    ROUND(AVG(oi.quantity * oi.item_list_price_at_sale)::NUMERIC, 2) AS avg_amount
FROM mipt.orders o
JOIN mipt.order_items oi ON o.order_id = oi.order_id
JOIN mipt.customer c ON o.customer_id = c.customer_id
GROUP BY c.customer_id
ORDER BY amount DESC, cnt_orders DESC
"""

df_4_groupby = pd.read_sql_query(text(query_4_groupby), engine)
print("GROUP BY version:")
display(df_4_groupby)

query_4_window =  """
WITH unique_orders AS (
    SELECT DISTINCT
        c.customer_id,
        o.order_id,
        oi.quantity * oi.item_list_price_at_sale AS item_amount
    FROM mipt.orders o
    JOIN mipt.order_items oi ON o.order_id = oi.order_id
    JOIN mipt.customer c ON o.customer_id = c.customer_id
)
SELECT DISTINCT
    customer_id,
    ROUND(SUM(item_amount) OVER (PARTITION BY customer_id)::NUMERIC, 2) AS amount,
    MAX(item_amount) OVER (PARTITION BY customer_id) AS max_amount,
    MIN(item_amount) OVER (PARTITION BY customer_id) AS min_amount,
    COUNT(order_id) OVER (PARTITION BY customer_id) AS cnt_orders,  -- Now counts unique orders
    ROUND(AVG(item_amount) OVER (PARTITION BY customer_id)::NUMERIC, 2) AS avg_amount
FROM unique_orders
ORDER BY amount DESC, cnt_orders DESC
"""

df_4_window = pd.read_sql_query(text(query_4_window), engine)
print("Window functions version:")
display(df_4_window)

GROUP BY version:


,customer_id,amount,max_amount,min_amount,cnt_orders,avg_amount
0,2183,136632.00,20056.60,1073.07,14,9759.46
1,1597,133657.00,20914.70,1720.70,12,11138.09
2,941,129790.00,20914.70,2115.02,10,12978.99
3,1129,129189.00,19773.60,1743.72,13,9937.65
4,637,109335.00,17796.24,360.40,13,8410.36
...,...,...,...,...,...,...
3488,784,796.21,748.17,48.04,2,398.10
3489,2089,598.00,416.98,181.02,2,299.00
3490,301,432.94,290.62,142.32,2,216.47
3491,2532,71.49,71.49,71.49,1,71.49


Window functions version:


,customer_id,amount,max_amount,min_amount,cnt_orders,avg_amount
0,2183,136632.00,20056.60,1073.07,14,9759.46
1,1597,133657.00,20914.70,1720.70,12,11138.09
2,941,129790.00,20914.70,2115.02,10,12978.99
3,1129,129189.00,19773.60,1743.72,13,9937.65
4,637,109335.00,17796.24,360.40,13,8410.36
...,...,...,...,...,...,...
3488,784,796.21,748.17,48.04,2,398.10
3489,2089,598.00,416.98,181.02,2,299.00
3490,301,432.94,290.62,142.32,2,216.47
3491,2532,71.49,71.49,71.49,1,71.49


In [7]:
# 5. Найти имена и фамилии клиентов с топ-3 минимальной и топ-3 максимальной суммой транзакций за весь период
# (учесть клиентов, у которых нет заказов, приняв их сумму транзакций за 0).
query_5 = """
WITH sums AS (
    SELECT
        c.customer_id,
        c.first_name,
        c.last_name,
        COALESCE(SUM(oi.quantity * oi.item_list_price_at_sale), 0) AS amount
    FROM mipt.customer c
    LEFT JOIN mipt.orders o ON c.customer_id = o.customer_id AND o.order_status = 'Approved'
    LEFT JOIN mipt.order_items oi ON o.order_id = oi.order_id
    GROUP BY c.customer_id, c.first_name, c.last_name
)
(SELECT first_name, last_name, amount
 FROM sums
 ORDER BY amount DESC
 LIMIT 3)
UNION
(SELECT first_name, last_name, amount
 FROM sums
 ORDER BY amount ASC
 LIMIT 3)
ORDER BY amount DESC
"""

df_5 = pd.read_sql_query(text(query_5), engine)
df_5

,first_name,last_name,amount
0,Jillie,Fyndon,136632.450
1,Jeffry,Slowly,133657.060
2,Tye,Doohan,129789.945
3,Elisha,Venny,0.000
4,Rodd,Spare,0.000
5,Shaylynn,Epsley,0.000


In [8]:
# 6. Вывести только вторые транзакции клиентов (если они есть) с помощью оконных функций.
# Если у клиента меньше двух транзакций, он не должен попасть в результат.
query_6 = """
SELECT
    customer_id,
    order_id,
    order_date
FROM (
    SELECT
        o.customer_id,
        o.order_id,
        o.order_date,
        ROW_NUMBER() OVER (PARTITION BY o.customer_id ORDER BY o.order_date) AS rn
    FROM mipt.orders o
    WHERE o.order_status = 'Approved'
) sub
WHERE rn = 2
ORDER BY customer_id
"""

df_6 = pd.read_sql_query(text(query_6), engine)
df_6

,customer_id,order_id,order_date
0,1,13424,2017-02-21
1,2,6743,2017-06-11
2,3,15188,2017-03-24
3,4,14648,2017-06-18
4,5,19993,2017-04-28
...,...,...,...
3439,3497,10493,2017-09-12
3440,3498,17866,2017-02-10
3441,3499,2433,2017-02-25
3442,3500,14870,2017-01-11


In [9]:
# 7. Вывести имена, фамилии и профессии клиентов, а также длительность максимального интервала (в днях) между двумя последовательными заказами.
# Исключить клиентов, у которых только один или меньше заказов.
query_7 = """
SELECT
    first_name,
    last_name,
    job_title,
    MAX(interval_days) AS max_interval
FROM (
    SELECT
        c.first_name,
        c.last_name,
        c.job_title,
        (LEAD(o.order_date::DATE) OVER (PARTITION BY c.customer_id ORDER BY o.order_date::DATE) - o.order_date::DATE) AS interval_days
    FROM mipt.customer c
    JOIN mipt.orders o ON c.customer_id = o.customer_id
    WHERE o.order_status = 'Approved'
) sub
WHERE interval_days IS NOT NULL
GROUP BY first_name, last_name, job_title
ORDER BY max_interval DESC
"""

df_7 = pd.read_sql_query(text(query_7), engine)
df_7

,first_name,last_name,job_title,max_interval
0,Susanetta,,Legal Assistant,357
1,Osborne,Nawton,Registered Nurse,337
2,Stoddard,Giacomoni,Structural Analysis Engineer,330
3,Royall,Terris,Geological Engineer,330
4,Gregorius,Cockram,Data Coordiator,330
...,...,...,...,...
3438,Ibrahim,Wibrew,,1
3439,Alie,Sowle,Budget/Accounting Analyst II,1
3440,Arch,Van der Kruis,Business Systems Development Analyst,1
3441,Consolata,Clacson,Geologist IV,1


In [10]:
# 8. Найти топ-5 клиентов (по общему доходу) в каждом сегменте благосостояния (wealth_segment).
# Вывести имя, фамилию, сегмент и общий доход. Если в сегменте менее 5 клиентов, вывести всех.
query_8 = """
WITH customer_amounts AS (
    SELECT
        c.customer_id,
        c.first_name,
        c.last_name,
        c.wealth_segment,
        SUM(oi.quantity * oi.item_list_price_at_sale) AS amount
    FROM mipt.customer c
    JOIN mipt.orders o ON c.customer_id = o.customer_id
    JOIN mipt.order_items oi ON o.order_id = oi.order_id
    WHERE o.order_status = 'Approved'
    GROUP BY c.customer_id, c.first_name, c.last_name, c.wealth_segment
)
SELECT
    first_name,
    last_name,
    wealth_segment,
    amount,
    rn
FROM (
    SELECT
        *,
        ROW_NUMBER() OVER (PARTITION BY wealth_segment ORDER BY amount DESC) AS rn
    FROM customer_amounts
) sub
WHERE rn <= 5
ORDER BY wealth_segment, rn
"""

df_8 = pd.read_sql_query(text(query_8), engine)
df_8

,first_name,last_name,wealth_segment,amount,rn
0,Jeffry,Slowly,Affluent Customer,133657.060,1
1,Tye,Doohan,Affluent Customer,129789.945,2
2,Herc,McIlhone,Affluent Customer,107476.690,3
3,Queenie,Flips,Affluent Customer,106182.330,4
4,Jessamine,Brazear,Affluent Customer,98618.770,5
5,Mercy,Wilsone,High Net Worth,109334.740,1
6,Lockwood,Exroll,High Net Worth,92405.180,2
7,Linell,,High Net Worth,91450.180,3
8,Gayelord,Lipman,High Net Worth,90493.060,4
9,Jonell,Gon,High Net Worth,87555.700,5
